In [2]:
from datasets import load_dataset, Dataset

In [10]:
hinmix_ds = load_dataset("kartikagg98/HINMIX_hi-en", "lcsalign-hicm", split="train", streaming=True)
hicm_set = hinmix_ds.take(7000)
hicm_set = list(hicm_set)

In [19]:
with open("hinmix_sample.txt", "w", encoding="utf-8") as f_txt:
    for sample in hicm_set:
        f_txt.write(sample["text"] + "\n")


In [12]:
import json

sentences = [sample["text"] for sample in hicm_set]

with open("hinmix_sample.json", "w", encoding="utf-8") as f_json:
    json.dump(sentences, f_json, ensure_ascii=False, indent=2)

In [ ]:
hinmix_dst_e = load_dataset("kartikagg98/HINMIX_hi-en", "lcsalign-en", split="test", streaming=True)
hinmix_dst_en = hinmix_dst_e.take(2500)
hinmix_dst_en = list(hinmix_dst_en)

In [17]:
hinmix_dst_h = load_dataset("kartikagg98/HINMIX_hi-en", "lcsalign-hi", split="test", streaming=True)
hinmix_dst_hi = hinmix_dst_h.take(2500)
hinmix_dst_hi = list(hinmix_dst_hi.take(2500))

In [20]:
with open("hinmix_sample.txt", "a", encoding="utf-8") as f_txt:
    for sample in hinmix_dst_en:
        f_txt.write(sample["text"] + "\n")

In [21]:
with open("hinmix_sample.txt", "a", encoding="utf-8") as f_txt:
    for sample in hinmix_dst_hi:
        f_txt.write(sample["text"] + "\n")

In [2]:
import sentencepiece as spm

In [25]:
file_path = "/home/angkul/my_data/coding/agi/hindi_GPT/hinmix_sample.txt"

In [ ]:
spm.SentencePieceTrainer.train(
    input=file_path,
    model_prefix='hinglish_32k',
    vocab_size=32000,
    model_type='bpe',
    user_defined_symbols=['[HI]', '[EN]'],
    split_digits=True,
    # allow_whitespace_only_pieces=True,
    add_dummy_prefix=False,
    remove_extra_whitespaces=True,
    normalization_rule_name='nfkc',
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3
)

In [3]:
# Load the trained model
sp = spm.SentencePieceProcessor()
sp.load('/home/angkul/my_data/coding/agi/hindi_GPT/hinglish_32k.model')

# Tokenize mixed text
text = "I read पुस्तकालय में books"
tokens = sp.encode_as_pieces(text)
ids = sp.encode_as_ids(text)

print("Tokens:", tokens)
print("Token IDs:", ids)

# Output example:
# Tokens: ['▁I', '▁read', '▁पुस्त', 'कालय', '▁में', '▁books']
# Token IDs: [10, 42, 1500, 1501, 2500, 99]

Tokens: ['I', '▁read', '▁पुस्तकालय', '▁में', '▁books']
Token IDs: [31954, 2401, 21934, 27, 6911]


In [4]:
sen = sp.Encode(text)
print(sen)
print(sp.Decode(sen))

[31954, 2401, 21934, 27, 6911]
I read पुस्तकालय में books


In [4]:
sp.GetPieceSize()

32000

In [4]:
from datasets import load_dataset

hindi_discourse = load_dataset('/home/angkul/my_data/coding/agi/hindi_GPT/hindi_discourse.py')
print(hindi_discourse)

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['Story_no', 'Sentence', 'Discourse Mode'],
        num_rows: 9968
    })
})


In [5]:
# hindi_discourse = hindi_discourse['train'][:5000]['Sentence']
hindi_discourse_subset = Dataset.from_dict((hindi_discourse))

In [7]:
tinystories = load_dataset('roneneldan/TinyStories', streaming=True)['train']
tinystories_subset = Dataset.from_list(list(tinystories.take(10000)))

In [8]:
print(type(tinystories_subset))
print(type(hindi_discourse_subset))
print((tinystories_subset))
print((hindi_discourse_subset))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text'],
    num_rows: 10000
})
Dataset({
    features: ['train'],
    num_rows: 9968
})


In [9]:
hindi_discourse_subset['train'][0]

{'Discourse Mode': 1,
 'Sentence': 'चेहरे पर इस की आँखें बहुत अजीब थीं।',
 'Story_no': 0}

In [10]:
# hindi_discourse_subset = hindi_discourse_subset.take(5000)

In [11]:
tinystories_texts = tinystories_subset['text']
hindi_sentences = [item['Sentence'] for item in hindi_discourse_subset['train']]

In [12]:
# combined_texts = [
#     {'text': text, 'Sentence': sentence}
#     for text, sentence in zip(tinystories_texts, hindi_sentences)
# ]

combined_texts = tinystories_texts + hindi_sentences

In [13]:
combined_dataset = Dataset.from_dict({'text': combined_texts})

In [14]:
len(combined_dataset)

19968

In [15]:
# Save the dataset to disk
combined_dataset.save_to_disk('combined_dataset')


Saving the dataset (0/1 shards):   0%|          | 0/19968 [00:00<?, ? examples/s]

In [16]:
combined_dataset['text'][5002]

'Johnny and his mom were in the park. Johnny skipped around the pathways, scanning for interesting things. He discovered a tiny bug, just a few centimeters long. He was very excited and picked it up carefully. As he watched it, the bug began to shrink! Johnny was amazed and asked his mom to explain. \n\n"The bug isn\'t really shrinking, Johnny," his mom said. "It\'s playing a trick on you. That\'s called camouflaging. It\'s shrinking its body to look like other things around it and protect itself." \n\n"It\'s kind of like when I have to hide behind the couch," Johnny said. \n\nHis mom laughed. "Exactly," she replied. "Animals are so clever. You have to be careful around them. We have to show them value and respect." \n\nJohnny looked at the bug intently. He was full of admiration and respect. "Okay," he said. "I promise to be careful."'